In [73]:
import numpy as np  
from sklearn.preprocessing import OneHotEncoder  
from torchvision import datasets  
from sklearn.utils import shuffle  

In [74]:
# Bước 1: Tải và chuẩn bị dữ liệu (giống như đã làm trước đó)  

trainset = datasets.CIFAR10(root='./data', train=True, download=True)  
class_to_fetch = 0  # Chọn lớp cụ thể  

# Sử dụng list comprehension để tối ưu hóa việc lọc và xử lý dữ liệu
X_train = np.array([np.array(img).reshape(-1) for img, label in trainset if label == class_to_fetch])
y_train = np.array([label for img, label in trainset if label == class_to_fetch])

# Chuẩn hóa dữ liệu
X_train = (X_train / 255.0 - 0.5) * 2  

# One-hot encoding cho nhãn
encoder = OneHotEncoder(sparse_output=False)  
y_train_onehot = encoder.fit_transform(y_train.reshape(-1, 1))  

# Trộn dữ liệu
X_train, y_train_onehot = shuffle(X_train, y_train_onehot, random_state=42)

Files already downloaded and verified


In [75]:
# Bước 2: Khởi tạo trọng số  
num_features = X_train.shape[1]  
num_classes = y_train_onehot.shape[1]  
weights = np.random.randn(num_features, num_classes) * 0.01  

In [76]:
def softmax(z):  
    exp_z = np.exp(z - np.max(z))  
    return exp_z / np.sum(exp_z)  

In [77]:
def cross_entropy_loss(y_true, y_pred):  
    return -np.sum(y_true * np.log(y_pred + 1e-15)) / y_true.shape[0]  

In [78]:
def model_forward(X, weights):  
    logits = np.dot(X, weights)  
    return softmax(logits)  


In [80]:
learning_rate = 0.01  
target_loss = 0.1  # Điều kiện dừng dựa trên loss
max_epochs = 1000  # Giới hạn số lần lặp để tránh lặp vô hạn
tolerance = 1e-6  # Độ chính xác để kiểm tra sự hội tụ

epoch = 0
previous_loss = float('inf')  # Khởi tạo giá trị loss trước đó

while True:
    
          
    y_pred = model_forward(X_train, weights)  
    loss = cross_entropy_loss(y_train_onehot, y_pred)  
    
    
    if loss < target_loss or epoch >= max_epochs or abs(previous_loss - loss) < tolerance:
        break   
    gradient = np.dot(X_train.T, (y_pred - y_train_onehot)) / X_train.shape[0]  
    
    weights -= learning_rate * gradient  
    previous_loss = loss
    epoch += 1  
    
    # # In thông tin
    # print(f"Epoch {epoch}, Loss: {loss:.6f}")

print(f"Training completed at Epoch {epoch}, Final Loss: {loss:.6f}")
print("Trọng số cuối cùng:", weights)


Training completed at Epoch 1, Final Loss: 34.531862
Trọng số cuối cùng: [[0.43140664]
 [0.83742177]
 [1.2728588 ]
 ...
 [0.05703914]
 [0.20508907]
 [0.25222452]]
